# Proyecto - Estudio del efecto de la profundidad y conexiones residuales en redes neuronales
### EL4106 Inteligencia Computacional

Profesor de Cátedra: Pablo Estévez<br>
Profesor Auxiliar: Pablo Cornejo<br>

### Tema: Regularización
La **regularización** es un conjunto de técnicas utilizadas en machine learning y deep learning para prevenir el **sobreajuste** (overfitting). El sobreajuste ocurre cuando un modelo aprende demasiado bien los detalles y ruido de los datos de entrenamiento, lo que resulta en un mal rendimiento cuando se evalúa en nuevos datos no vistos (conjunto de validación o test). La regularización ayuda a mejorar la **capacidad de generalización** del modelo, es decir, su capacidad para funcionar bien en datos no vistos.

### Tipos comunes de regularización:

1. **L2 Regularization (Weight Decay)**:
   - **Descripción**: Penaliza los pesos grandes al agregar un término a la función de pérdida que aumenta con el tamaño de los pesos.
   - **Fórmula**: El término de penalización en la función de pérdida es:
$ L(\theta) = L_0 + \lambda \sum_{i=1}^{n} \theta_i^2 $
     Donde:
     - $ L_0 $ es la pérdida original (por ejemplo, la entropía cruzada).
     - $ \theta_i $ son los pesos de la red.
     - $ \lambda $ es el hiperparámetro que controla la magnitud de la penalización (también llamado **weight decay**).
   - **Efecto**: Evita que los pesos crezcan demasiado, lo que podría hacer que el modelo sea demasiado dependiente de características particulares de los datos de entrenamiento.

2. **L1 Regularization**:
   - **Descripción**: Similar a L2, pero en lugar de penalizar los pesos grandes con un término cuadrático, penaliza la suma de los valores absolutos de los pesos.
   - **Fórmula**:
     $$
  L(\theta) = L_0 + \lambda \sum_{i=1}^{n} |\theta_i|
     $$
   - **Efecto**: Alienta la **sparsity** (la mayoría de los pesos se vuelven cero), lo que puede resultar en modelos más simples.

3. **Dropout**:
   - **Descripción**: Durante el entrenamiento, **desactiva aleatoriamente** una fracción de las neuronas en cada capa en cada paso de entrenamiento. Esto evita que el modelo se vuelva demasiado dependiente de cualquier neurona individual.
   - **Efecto**: Dropout actúa como un ensamble implícito de varias redes neuronales, entrenando diferentes subredes.

4. **Early Stopping**:
   - **Descripción**: Se detiene el entrenamiento del modelo cuando la precisión en el conjunto de validación deja de mejorar después de un cierto número de épocas.
   - **Efecto**: Evita que el modelo continúe ajustándose demasiado a los datos de entrenamiento.

5. **Data Augmentation**:
   - **Descripción**: **Aumentar artificialmente** el conjunto de datos al aplicar transformaciones aleatorias (rotaciones, escalado, recortes, etc.) puede hacer que el modelo vea más variedad en los datos y evitar el sobreajuste.
   - **Efecto**: Hace que el modelo aprenda características más generales y robustas.

6. **Batch Normalization**:
   - **Descripción**: Normaliza las activaciones de cada capa durante el entrenamiento, asegurando que la distribución de las activaciones permanezca estable. También tiene un efecto regularizador.
   - **Efecto**: Ayuda a estabilizar y acelerar el entrenamiento, además de reducir el sobreajuste.

### Aspectos a considerar al aplicar regularización:

1. **Hiperparámetros**:
   - El valor del coeficiente de regularización $\lambda$ en L2/L1 es crucial. Si es muy bajo, no se sentirá el efecto de la regularización, y si es muy alto, puede hacer que el modelo sea demasiado conservador.
   - Para **dropout**, la probabilidad de "dropout" $p$ debe seleccionarse cuidadosamente (valores comunes entre **0.3 y 0.5**).

2. **Balance entre regularización y ajuste**:
   - La regularización busca reducir el sobreajuste, pero demasiada regularización puede causar subajuste. Es importante encontrar un equilibrio.

3. **Evaluación con conjunto de validación**:
   - Siempre evalúa el impacto de la regularización en un conjunto de validación. Si la precisión de entrenamiento es alta pero la de validación es baja, probablemente estés sobreajustando.

4. **Tamaño del modelo y datos**:
   - Modelos grandes entrenados en conjuntos de datos pequeños son más propensos al sobreajuste, por lo que necesitan más regularización.

### Ejemplo práctico:

Si se está entrenando una red neuronal profunda en CIFAR-10 y se nota que el modelo tiene una alta precisión en el conjunto de entrenamiento pero baja en validación, es posible que esté ocurriendo sobreajuste. Por esto, se podria intentar:
- Agregar **L2 regularization** al optimizador ajustando el parámetro `weight_decay` (por ejemplo, **1e-4**).
- Aumentar el tamaño del conjunto de datos mediante **data augmentation** aplicando rotaciones, traslaciones o recortes aleatorios.

In [ ]:
!pip install umap-learn

In [ ]:
import os
import time
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import umap.umap_ as umap
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import sys
from sklearn.manifold import TSNE
from torch import Tensor
from typing import Type


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## Carga Datasets

In [ ]:
# Transformaciones para entrenamiento y validación
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),  # Recorte aleatorio después del padding
    torchvision.transforms.RandomHorizontalFlip(),      # Volteo horizontal aleatorio
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

val_test_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

# Cargar datasets CIFAR-10 con transformaciones
train_cifar10 = torchvision.datasets.CIFAR10(
    root="./cifar10",
    train=True,
    download=True,
    transform=train_transform  # Se aplica data augmentation aquí
)

test_cifar10 = torchvision.datasets.CIFAR10(
    root="./cifar10",
    train=False,
    download=True,
    transform=val_test_transform  # Solo normalización para testing
)

# Dividir dataset de entrenamiento y validación
train_cifar10, _ = torch.utils.data.random_split(train_cifar10, [45000, 5000], generator=torch.Generator().manual_seed(42))
_, val_cifar10 = torch.utils.data.random_split(
    torchvision.datasets.CIFAR10(root="./cifar10", train=True, transform=val_test_transform),
    [45000, 5000], generator=torch.Generator().manual_seed(42)
)

## Modelos

### RedPlana20

In [ ]:
# Definición de un bloque básico de la red plana (simple convolución + BN + ReLU)
class PlainBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(PlainBlock, self).__init__()
        self.conv = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(planes)

    def forward(self, x):
        out = F.relu(self.bn(self.conv(x)))
        return out

# Definición de la red plana con 20 capas convolucionales
class PlainNet20(nn.Module):
    def __init__(self, num_classes=10):
        super(PlainNet20, self).__init__()
        self.in_planes = 64

        # Capa de convolución inicial (1 capa)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # 20 capas en total, distribuidas en 3 layers
        self.layer1 = self._make_layer(64, 6, stride=1)   # 6 capas convolucionales (32x32)
        self.layer2 = self._make_layer(128, 7, stride=2)  # 7 capas convolucionales (16x16)
        self.layer3 = self._make_layer(256, 6, stride=2)  # 6 capas convolucionales (8x8)

        # Pooling global
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Capa completamente conectada
        self.fc = nn.Linear(256, num_classes)

    # Crear capas usando bloques básicos
    def _make_layer(self, planes, num_blocks, stride):
        layers = []
        strides = [stride] + [1] * (num_blocks - 1)
        for s in strides:
            layers.append(PlainBlock(self.in_planes, planes, stride=s))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):

        out = F.relu(self.bn1(self.conv1(x)))
        feature_maps['ReLUconv1'] = out
        out = self.layer1(out)
        out = self.layer2(out)
        feature_maps['Layer2'] = out
        out = self.layer3(out)
        feature_maps['Layer3'] = out
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

# Crear el modelo
model = PlainNet20(num_classes=10)

# Imprimir la arquitectura
print(model)

for name, param in model.named_parameters():
    print(name)

### RedPlana32

In [ ]:
# Definición de un bloque básico de la red plana (simple convolución + BN + ReLU)
class PlainBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(PlainBlock, self).__init__()
        self.conv = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(planes)

    def forward(self, x):
        out = F.relu(self.bn(self.conv(x)))
        return out

# Definición de la red plana con 32 capas convolucionales
class PlainNet32(nn.Module):
    def __init__(self, num_classes=10):
        super(PlainNet32, self).__init__()
        self.in_planes = 64

        # Capa de convolución inicial
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # 32 capas de convoluciones simples (sin conexiones residuales)
        self.layer1 = self._make_layer(64, 5, stride=1)   # 5 bloques (32x32)
        self.layer2 = self._make_layer(128, 5, stride=2)  # 5 bloques (16x16)
        self.layer3 = self._make_layer(256, 11, stride=2) # 11 bloques (8x8)
        self.layer4 = self._make_layer(512, 11, stride=2) # 11 bloques (4x4)

        # Pooling global
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Capa completamente conectada
        self.fc = nn.Linear(512, num_classes)

    # Crear capas usando bloques básicos
    def _make_layer(self, planes, num_blocks, stride):
        layers = []
        strides = [stride] + [1] * (num_blocks - 1)
        for s in strides:
            layers.append(PlainBlock(self.in_planes, planes, stride=s))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):

        out = F.relu(self.bn1(self.conv1(x)))
        feature_maps['ReLUconv1'] = out
        out = self.layer1(out)
        out = self.layer2(out)
        feature_maps['Layer2'] = out
        out = self.layer3(out)
        out = self.layer4(out)
        feature_maps['Layer4'] = out
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

# Crear el modelo
model = PlainNet32(num_classes=10)

# Imprimir la arquitectura
print(model)

for name, param in model.named_parameters():
    print(name)

### ResNet20

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))  # Apply BN and then ReLU
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # Apply shortcut connection
        out = F.relu(out)
        return out


class ResNet20(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet20, self).__init__()
        self.in_planes = 16  # Following CIFAR-10 architecture, we start with 16 filters

        # Initial convolutional layer (3x3 convolutions, per the description)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()

        # Residual layers
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)  # For 32x32 feature maps
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)  # For 16x16 feature maps
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)  # For 8x8 feature maps

        # Global average pooling and fully connected layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(64 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # Apply stride to the first block
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, planes, s))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # Initial convolution
        out = F.relu(self.bn1(self.conv1(x)))
        feature_maps['ReLUconv1'] = out
        # Residual layers
        out = self.layer1(out)
        out = self.layer2(out)
        feature_maps['Layer2'] = out
        out = self.layer3(out)
        feature_maps['Layer3'] = out

        # Global average pooling
        out = self.avgpool(out)
        out = torch.flatten(out, 1)

        # Fully connected layer
        out = self.linear(out)
        return out

# Create the ResNet-20 model
def ResNet_20():
    return ResNet20(BasicBlock, [3, 3, 3])  # Each number represents 2n layers, so 3 blocks for each feature map size

# Instantiate and print the model to verify its structure
model = ResNet_20()

# Print the model structure
print(model)

# Print the model structure
print(model)
shortcut_count = 0
for name, module in model.named_modules():
    if isinstance(module, BasicBlock):
        shortcut_count += 1

print(f"Total de shortcuts en el modelo: {shortcut_count}")

for name, param in model.named_parameters():
    print(name)

### ResNet32

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        # Conexión de atajo
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        # Función residual
        out = F.relu(self.bn1(self.conv1(x)))  # Aplica BN y luego ReLU
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# Modificar el modelo para devolver los embeddings de todas las capas
class ResNet32(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet32, self).__init__()
        self.in_planes = 64

        # Capa de convolución inicial
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()

        # Capas residuales
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # Pooling global y capa completamente conectada
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, planes, s))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # Convolución inicial
        out = self.relu(self.bn1(self.conv1(x)))
        feature_maps['ReLUconv1'] = out
        out = self.layer1(out)
        out = self.layer2(out)
        feature_maps['Layer2'] = out
        out = self.layer3(out)
        out = self.layer4(out)
        feature_maps['Layer4'] = out
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        # Capa completamente conectada
        out = self.linear(out)
        return out

def ResNet_32():
    return ResNet32(BasicBlock, [3, 4, 6, 2])

model = ResNet_32()
for name, param in model.named_parameters():
    print(name)

shortcut_count = 0
for name, module in model.named_modules():
    if isinstance(module, BasicBlock):
        shortcut_count += 1

print(f"Total de shortcuts en el modelo: {shortcut_count}")


## Funciones para Entrenamiento

In [ ]:
def show_gradients_Red_Plana_20(model):
    layers_to_show = ['conv1', 'bn1', 'layer2.2.conv', 'layer2.2.bn', 'fc']  # Capas inicial, intermedia y final
    for name, param in model.named_parameters():
        if any(layer in name for layer in layers_to_show) and param.requires_grad and param.grad is not None:
            grad = param.grad.cpu().numpy()
            print(f"Gradientes para {name}: min={grad.min()}, max={grad.max()}, mean={grad.mean()}, std={grad.std()}")
            plt.figure(figsize=(8, 6))
            plt.hist(grad.flatten(), bins=50)
            plt.title(f'Gradientes para {name}')
            plt.xlabel('Valor del gradiente')
            plt.ylabel('Frecuencia')
            plt.show()

def show_gradients_Red_Plana_32(model):
    layers_to_show = ['conv1', 'bn1', 'layer3.0', 'fc']  # Capas inicial, intermedia y final
    for name, param in model.named_parameters():
        if any(layer in name for layer in layers_to_show) and param.requires_grad and param.grad is not None:
            grad = param.grad.cpu().numpy()
            print(f"Gradientes para {name}: min={grad.min()}, max={grad.max()}, mean={grad.mean()}, std={grad.std()}")
            plt.figure(figsize=(8, 6))
            plt.hist(grad.flatten(), bins=50)
            plt.title(f'Gradientes para {name}')
            plt.xlabel('Valor del gradiente')
            plt.ylabel('Frecuencia')
            plt.show()


def show_gradients_ResNet_20(model):
    layers_to_show = ['conv1', 'bn1', 'layer2.1.conv1', 'layer2.1.bn1', 'linear']  # Capas inicial, intermedia y final
    for name, param in model.named_parameters():
        if any(layer in name for layer in layers_to_show) and param.requires_grad and param.grad is not None:
            grad = param.grad.cpu().numpy()
            print(f"Gradientes para {name}: min={grad.min()}, max={grad.max()}, mean={grad.mean()}, std={grad.std()}")
            plt.figure(figsize=(8, 6))
            plt.hist(grad.flatten(), bins=50)
            plt.title(f'Gradientes para {name}')
            plt.xlabel('Valor del gradiente')
            plt.ylabel('Frecuencia')
            plt.show()

def show_gradients_ResNet_32(model):
    layers_to_show = ['conv1', 'bn1', 'layer2.1.conv1', 'layer2.1.bn1', 'linear']  # Capas inicial, intermedia y final
    for name, param in model.named_parameters():
        if any(layer in name for layer in layers_to_show) and param.requires_grad and param.grad is not None:
            grad = param.grad.cpu().numpy()
            print(f"Gradientes para {name}: min={grad.min()}, max={grad.max()}, mean={grad.mean()}, std={grad.std()}")
            plt.figure(figsize=(8, 6))
            plt.hist(grad.flatten(), bins=50)
            plt.title(f'Gradientes para {name}')
            plt.xlabel('Valor del gradiente')
            plt.ylabel('Frecuencia')
            plt.show()

In [ ]:

# Initialize He weights
def initialize_weights_he(model):
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, a=0, mode='fan_out', nonlinearity='relu')
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)


def show_curves(curves):
    fig, ax = plt.subplots(1, 2, figsize=(13, 5))
    fig.set_facecolor('white')

    # Asegúrate de que los datos estén en la CPU antes de convertirlos a NumPy
    epochs = np.arange(len(curves["val_loss"])) + 1

    ax[0].plot(epochs, np.array(curves['val_loss']), label='validation')
    ax[0].plot(epochs, np.array(curves['train_loss']), label='training')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Loss')
    ax[0].set_title('Loss evolution during training')
    ax[0].legend()

    ax[1].plot(epochs, np.array(curves['val_acc']), label='validation')
    ax[1].plot(epochs, np.array(curves['train_acc']), label='training')
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_title('Accuracy evolution during training')
    ax[1].legend()

    plt.tight_layout()
    plt.savefig('training_curves.png')
    plt.show()

# Visualize feature maps
def show_feature_maps(feature_maps):
    for layer_name, feature_map in feature_maps.items():
        fmap = feature_map[0].cpu().numpy()  # Convert to CPU and NumPy for visualization
        num_feature_maps = fmap.shape[0]

        # Plot a grid of feature maps (first 8 feature maps)
        fig, axes = plt.subplots(1, min(8, num_feature_maps), figsize=(20, 5))
        fig.suptitle(f"Feature Maps from Layer {layer_name}", fontsize=16)

        for i in range(min(8, num_feature_maps)):
            axes[i].imshow(fmap[i], cmap='viridis')
            axes[i].axis('off')

        plt.show()

def checkpoint_save(model, optimizer, epoch, filename):
    checkpoint_data = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint_data, os.path.join(checkpoint_dir, filename))
    print(f"Checkpoint guardado en {os.path.join(checkpoint_dir, filename)}")

def checkpoint_resume(model, optimizer, filename):
    checkpoint_path = os.path.join(checkpoint_dir, filename)
    if os.path.isfile(checkpoint_path):
        checkpoint_data = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint_data['model_state_dict'])
        optimizer.load_state_dict(checkpoint_data['optimizer_state_dict'])
        epoch = checkpoint_data['epoch']
        print(f"Checkpoint cargado desde '{checkpoint_path}' (época {epoch})")
        return epoch
    else:
        print(f"No se encontró ningún checkpoint en '{checkpoint_path}'")
        return None

# Training step function
def train_step(x_batch, y_batch, model, optimizer, criterion, device):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    y_predicted = model(x_batch)
    loss = criterion(y_predicted, y_batch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return y_predicted, loss

# Evaluation function
def evaluate(val_loader, model, criterion, device):
    model.eval()
    cumulative_loss = 0
    cumulative_corrects = 0
    data_count = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            final_output = model(imgs)
            loss = criterion(final_output, labels)
            cumulative_loss += loss.item() * len(labels)
            data_count += len(labels)
            _, pred_class = final_output.max(1)
            cumulative_corrects += (pred_class == labels).sum().item()
    val_acc = cumulative_corrects / data_count
    val_loss = cumulative_loss / data_count
    return val_acc, val_loss

#Segunda Resnet
def train_model(
    model,
    train_dataset,
    val_dataset,
    epochs,
    max_iterations,
    criterion,
    batch_size,
    lr,
    weight_decay,
    n_evaluations_per_epoch,
    early_stop_thresh,  # Early stopping threshold
    show_gradients,
    patience,
    use_gpu=True,
    data_augmentation=False,
    resume_checkpoint=None
):
    original_transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ])

    if data_augmentation:
        train_dataset.dataset.transform = torchvision.transforms.Compose([
            torchvision.transforms.RandomCrop(32, padding=4),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
        ])
    else:
        train_dataset.dataset.transform = original_transform

    print(f"Using train transform: {train_dataset.dataset.transform}")
    print(f"Using validation transform: {val_dataset.dataset.transform}")


    # Usar GPU si está disponible
    device = 'cuda:0'#torch.device('cuda' if use_gpu else 'cpu')
    model = model.to(device)

    # DataLoaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=use_gpu)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, pin_memory=use_gpu)

    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=patience, threshold=0.0001, threshold_mode='abs')
    scaler = torch.amp.GradScaler()

    curves = {"train_acc": [], "val_acc": [], "train_loss": [], "val_loss": []}
    t0 = time.perf_counter()
    iteration = 0
    n_batches = len(train_loader)
    start_epoch = 0
    if resume_checkpoint is not None:
        start_epoch = checkpoint_resume(model, optimizer, resume_checkpoint)
        print(f"Reanudando desde la época {start_epoch}")

    best_val_loss = float('inf')
    best_epoch = -1
    early_stop_counter = 0

    for epoch in range(start_epoch, epochs):
        print(optimizer.param_groups[0]["lr"])
        print(f"\rEpoch {epoch + 1}/{epochs}")
        cumulative_train_loss = 0
        cumulative_train_corrects = 0
        train_loss_count = 0
        train_acc_count = 0

        model.train()
        for i, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_predicted, loss = train_step(x_batch, y_batch, model, optimizer, criterion, device)

            cumulative_train_loss += loss.item()
            train_loss_count += 1
            train_acc_count += y_batch.shape[0]

            # Accuracy calculation
            class_prediction = torch.argmax(y_predicted, axis=1).long()
            cumulative_train_corrects += (y_batch == class_prediction).sum().item()


            # Registro de métricas
            if (i + 1) % (n_batches // n_evaluations_per_epoch) == 0:
                train_loss = cumulative_train_loss / train_loss_count
                train_acc = cumulative_train_corrects / train_acc_count

                print(
                    f"Iteración {iteration + 1} - Lote {i + 1}/{n_batches} - "
                    f"Pérdida de Entrenamiento: {train_loss:.4f}, Precisión de Entrenamiento: {train_acc:.4f}"
                )


            iteration += 1
            if iteration >= max_iterations:
                print(f"Número máximo de iteraciones alcanzado: {max_iterations}")
                break

        val_acc, val_loss = evaluate(val_loader, model, criterion, device)
        print(f"Val loss: {val_loss:.4f}, Val acc: {val_acc:.4f}")

        train_loss = cumulative_train_loss / train_loss_count
        train_acc = cumulative_train_corrects / train_acc_count

        curves["train_acc"].append(train_acc)
        curves["val_acc"].append(val_acc)
        curves["train_loss"].append(train_loss)
        curves["val_loss"].append(val_loss)
        scheduler.step(val_loss)

        # Checkpointing the best model based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            early_stop_counter = 0
            checkpoint_filename = f"best_checkpoint_epoch_{epoch + 1}.pth"
            checkpoint_save(model, optimizer, epoch, checkpoint_filename)
            print(f"Checkpoint del mejor modelo guardado en la época {epoch + 1}")
        else:
            early_stop_counter += 1

        if epoch + 1 == 1 or (epoch + 1) % 5 == 0 or early_stop_counter >= early_stop_thresh:
            show_gradients(model)
            show_feature_maps(feature_maps)

        if early_stop_counter >= early_stop_thresh:
            print(f"Early stopping at epoch {epoch + 1}")
            break

        if iteration >= max_iterations:
            break

    total_time = time.perf_counter() - t0
    print(f"\nTiempo total de entrenamiento: {total_time:.2f} segundos")

    # Ensure the model is on CPU after training
    model.cpu()

    if data_augmentation:
        train_dataset.dataset.transform = original_transform

    return curves

## Entrenamiento Distintos Modelos

### RedPlana20

In [ ]:
use_gpu = True

# Checkpointing functions
checkpoint_dir = "/content/drive/MyDrive/CheckpointsRedPlana20"
os.makedirs(checkpoint_dir, exist_ok=True)

# Initialize feature map dictionary
feature_maps = {}
if __name__ == "__main__":
    # Hiperparámetros
    batch_size = 128
    epochs = 70
    max_iterations = 600000  # Ajusta según sea necesario
    learning_rate = 0.1
    n_evaluations_per_epoch = 10
    use_gpu = torch.cuda.is_available()
    data_augmentation = True
    weight_decay = 0.0001
    early_stop_thresh = 15
    patience = 3

    # Inicializar el modelo
    model = PlainNet20(num_classes=10)
    initialize_weights_he(model)
    print(model)

    # Definir función de pérdida
    criterion = nn.CrossEntropyLoss()

    # Entrenar el modelo
    curves = train_model(
        model=model,
        train_dataset=train_cifar10,
        val_dataset=val_cifar10,
        epochs=epochs,
        max_iterations=max_iterations,
        criterion=criterion,
        batch_size=batch_size,
        lr=learning_rate,
        n_evaluations_per_epoch=n_evaluations_per_epoch,
        early_stop_thresh=early_stop_thresh,
        show_gradients=show_gradients_Red_Plana_20,
        patience=patience,
        use_gpu=use_gpu,
        data_augmentation=data_augmentation,
        resume_checkpoint=None,  # Establece a una cadena de caracteres para reanudar, por ejemplo, "best_checkpoint_epoch_10.pth"
        weight_decay=weight_decay
    )

    # Plotear curvas de entrenamiento
    show_curves(curves)


In [ ]:
use_gpu = True
device = torch.device('cuda' if use_gpu else 'cpu')
model = model.to(device)

# Obtener embeddings y realizar visualizaciones
def get_embeddings(model, dataloader, use_gpu):
    model.eval()  # Establecer el modelo en modo evaluación
    device = torch.device('cuda' if use_gpu else 'cpu')
    embeddings_list = []
    labels_list = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Pasar los datos a través del modelo hasta antes de la capa totalmente conectada
            out = F.relu((model.bn1(model.conv1(images))))
            out = model.layer1(out)
            out = model.layer2(out)
            out = model.layer3(out)
            out = model.avgpool(out)
            out = out.view(out.size(0), -1)  # Aplanar las características

            embeddings_list.append(out.cpu())
            labels_list.append(labels.cpu())

    embeddings = torch.cat(embeddings_list)
    labels = torch.cat(labels_list)
    return embeddings, labels


# Crear un dataloader para todo el conjunto de entrenamiento
full_train_loader = torch.utils.data.DataLoader(
    train_cifar10, batch_size=128, shuffle=False, num_workers=2, pin_memory=use_gpu
)

# Obtener las embeddings
embeddings, labels = get_embeddings(model, full_train_loader, use_gpu)
# Convertir embeddings y labels a NumPy arrays
embeddings_np = embeddings.numpy()
labels_np = labels.numpy()

# Visualización con t-SNE (Scikit-learn)
from sklearn.manifold import TSNE

# Configurar t-SNE
tsne = TSNE(n_components=2, perplexity=50, n_iter=1000, random_state=42)

# Ajustar y transformar las embeddings
embeddings_tsne = tsne.fit_transform(embeddings_np)

# Graficar
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=embeddings_tsne[:, 0], y=embeddings_tsne[:, 1],
    hue=labels_np, palette=sns.color_palette("hls", 10),
    legend="full", alpha=0.5
)
plt.title('t-SNE Dimensionality Reduction')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend(title='Classes')
plt.show()

# Visualización con UMAP
import umap.umap_ as umap

# Configurar UMAP
umap_reducer = umap.UMAP(
    n_neighbors=30,
    min_dist=0.1,
    n_components=2,
    metric='euclidean',
    random_state=42
)

# Ajustar y transformar las embeddings
embeddings_umap = umap_reducer.fit_transform(embeddings_np)

# Graficar
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=embeddings_umap[:, 0], y=embeddings_umap[:, 1],
    hue=labels_np, palette=sns.color_palette("hls", 10),
    legend="full", alpha=0.5
)
plt.title('UMAP Dimensionality Reduction')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend(title='Classes')
plt.show()

### RedPlana32

In [ ]:
use_gpu = True

# Checkpointing functions
checkpoint_dir = "/content/drive/MyDrive/CheckpointsRedPlana32"
os.makedirs(checkpoint_dir, exist_ok=True)

# Initialize feature map dictionary
feature_maps = {}
if __name__ == "__main__":
    # Hiperparámetros
    batch_size = 128
    epochs = 70
    max_iterations = 600000  # Ajusta según sea necesario
    learning_rate = 0.1
    n_evaluations_per_epoch = 10
    use_gpu = torch.cuda.is_available()
    data_augmentation = True
    weight_decay = 0.0001
    early_stop_thresh = 15
    patience = 3

    # Inicializar el modelo
    model = PlainNet32(num_classes=10)
    initialize_weights_he(model)
    print(model)

    # Definir función de pérdida
    criterion = nn.CrossEntropyLoss()

    # Entrenar el modelo
    curves = train_model(
        model=model,
        train_dataset=train_cifar10,
        val_dataset=val_cifar10,
        epochs=epochs,
        max_iterations=max_iterations,
        criterion=criterion,
        batch_size=batch_size,
        lr=learning_rate,
        n_evaluations_per_epoch=n_evaluations_per_epoch,
        early_stop_thresh=early_stop_thresh,
        show_gradients=show_gradients_Red_Plana_20,
        patience=patience,
        use_gpu=use_gpu,
        data_augmentation=data_augmentation,
        resume_checkpoint=None,
        weight_decay=weight_decay
    )

    # Plotear curvas de entrenamiento
    show_curves(curves)


In [ ]:
use_gpu = True
device = torch.device('cuda' if use_gpu else 'cpu')
model = model.to(device)

# Obtener embeddings y realizar visualizaciones
def get_embeddings(model, dataloader, use_gpu):
    model.eval()  # Establecer el modelo en modo evaluación
    device = torch.device('cuda' if use_gpu else 'cpu')
    embeddings_list = []
    labels_list = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Pasar los datos a través del modelo hasta antes de la capa totalmente conectada
            out = F.relu(model.bn1(model.conv1(images)))
            out = model.layer1(out)
            out = model.layer2(out)
            out = model.layer3(out)
            out = model.layer4(out)
            out = model.avgpool(out)
            out = out.view(out.size(0), -1)  # Aplanar las características

            embeddings_list.append(out.cpu())
            labels_list.append(labels.cpu())

    embeddings = torch.cat(embeddings_list)
    labels = torch.cat(labels_list)
    return embeddings, labels


# Crear un dataloader para todo el conjunto de entrenamiento
full_train_loader = torch.utils.data.DataLoader(
    train_cifar10, batch_size=128, shuffle=False, num_workers=2, pin_memory=use_gpu
)

# Obtener las embeddings
embeddings, labels = get_embeddings(model, full_train_loader, use_gpu)
# Convertir embeddings y labels a NumPy arrays
embeddings_np = embeddings.numpy()
labels_np = labels.numpy()

# Visualización con t-SNE (Scikit-learn)
from sklearn.manifold import TSNE

# Configurar t-SNE
tsne = TSNE(n_components=2, perplexity=50, n_iter=1000, random_state=42)

# Ajustar y transformar las embeddings
embeddings_tsne = tsne.fit_transform(embeddings_np)

# Graficar
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=embeddings_tsne[:, 0], y=embeddings_tsne[:, 1],
    hue=labels_np, palette=sns.color_palette("hls", 10),
    legend="full", alpha=0.5
)
plt.title('t-SNE Dimensionality Reduction')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend(title='Classes')
plt.show()

# Visualización con UMAP
import umap.umap_ as umap

# Configurar UMAP
umap_reducer = umap.UMAP(
    n_neighbors=30,
    min_dist=0.1,
    n_components=2,
    metric='euclidean',
    random_state=42
)

# Ajustar y transformar las embeddings
embeddings_umap = umap_reducer.fit_transform(embeddings_np)

# Graficar
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=embeddings_umap[:, 0], y=embeddings_umap[:, 1],
    hue=labels_np, palette=sns.color_palette("hls", 10),
    legend="full", alpha=0.5
)
plt.title('UMAP Dimensionality Reduction')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend(title='Classes')
plt.show()

### Resnet20

In [ ]:
use_gpu = True

# Checkpointing functions
checkpoint_dir = "/content/drive/MyDrive/CheckpointsResNet20"
os.makedirs(checkpoint_dir, exist_ok=True)

# Initialize feature map dictionary
feature_maps = {}
if __name__ == "__main__":
    # Hiperparámetros
    batch_size = 128
    epochs = 70
    max_iterations = 600000  # Ajusta según sea necesario
    learning_rate = 0.1
    n_evaluations_per_epoch = 10
    use_gpu = torch.cuda.is_available()
    data_augmentation = True
    weight_decay = 0.0001
    early_stop_thresh = 15
    patience = 3

    # Inicializar el modelo
    model = ResNet_20()
    initialize_weights_he(model)
    print(model)

    # Definir función de pérdida
    criterion = nn.CrossEntropyLoss()

    # Entrenar el modelo
    curves = train_model(
        model=model,
        train_dataset=train_cifar10,
        val_dataset=val_cifar10,
        epochs=epochs,
        max_iterations=max_iterations,
        criterion=criterion,
        batch_size=batch_size,
        lr=learning_rate,
        n_evaluations_per_epoch=n_evaluations_per_epoch,
        early_stop_thresh=early_stop_thresh,
        show_gradients=show_gradients_ResNet_20,
        patience=patience,
        use_gpu=use_gpu,
        data_augmentation=data_augmentation,
        resume_checkpoint=None,  # Establece a una cadena de caracteres para reanudar, por ejemplo, "best_checkpoint_epoch_10.pth"
        weight_decay=weight_decay
    )

    # Plotear curvas de entrenamiento
    show_curves(curves)


In [ ]:
use_gpu = True
device = torch.device('cuda' if use_gpu else 'cpu')
model = model.to(device)

# Obtener embeddings y realizar visualizaciones
def get_embeddings(model, dataloader, use_gpu):
    model.eval()  # Establecer el modelo en modo evaluación
    device = torch.device('cuda' if use_gpu else 'cpu')
    embeddings_list = []
    labels_list = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Pasar los datos a través del modelo hasta antes de la capa totalmente conectada
            out = model.relu(model.bn1(model.conv1(images)))
            out = model.layer1(out)
            out = model.layer2(out)
            out = model.layer3(out)
            out = model.avgpool(out)
            out = out.view(out.size(0), -1)  # Aplanar las características

            embeddings_list.append(out.cpu())
            labels_list.append(labels.cpu())

    embeddings = torch.cat(embeddings_list)
    labels = torch.cat(labels_list)
    return embeddings, labels


# Crear un dataloader para todo el conjunto de entrenamiento
full_train_loader = torch.utils.data.DataLoader(
    train_cifar10, batch_size=128, shuffle=False, num_workers=2, pin_memory=use_gpu
)

# Obtener las embeddings
embeddings, labels = get_embeddings(model, full_train_loader, use_gpu)
# Convertir embeddings y labels a NumPy arrays
embeddings_np = embeddings.numpy()
labels_np = labels.numpy()

# Visualización con t-SNE (Scikit-learn)
from sklearn.manifold import TSNE

# Configurar t-SNE
tsne = TSNE(n_components=2, perplexity=50, n_iter=1000, random_state=42)

# Ajustar y transformar las embeddings
embeddings_tsne = tsne.fit_transform(embeddings_np)

# Graficar
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=embeddings_tsne[:, 0], y=embeddings_tsne[:, 1],
    hue=labels_np, palette=sns.color_palette("hls", 10),
    legend="full", alpha=0.5
)
plt.title('t-SNE Dimensionality Reduction')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend(title='Classes')
plt.show()

# Visualización con UMAP
import umap.umap_ as umap

# Configurar UMAP
umap_reducer = umap.UMAP(
    n_neighbors=30,
    min_dist=0.1,
    n_components=2,
    metric='euclidean',
    random_state=42
)

# Ajustar y transformar las embeddings
embeddings_umap = umap_reducer.fit_transform(embeddings_np)

# Graficar
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=embeddings_umap[:, 0], y=embeddings_umap[:, 1],
    hue=labels_np, palette=sns.color_palette("hls", 10),
    legend="full", alpha=0.5
)
plt.title('UMAP Dimensionality Reduction')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend(title='Classes')
plt.show()